In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os


In [2]:
os.path.dirname(os.getcwd())

'/Users/antonioricciardi/projects/rl_relrepr_gymnasium'

In [3]:
# change working directory to the root of the project
os.chdir(os.path.dirname(os.getcwd()))

test_path = 'experiments/stitching_tests'

Preview data

In [4]:
env_id = "CarRacing-v2-multicolor"
env_id_encoder = "CarRacing-v2"
env_info = "rgb"

""" SHOW LOGGED DATA """

# read stitching_results_relative_ppo with pandas
df = pd.read_csv(f'{test_path}/{env_id}/{env_info}/absolute/{env_id_encoder}_relu_stitching_results_ppo.csv', sep=',')
df.tail()

,env_seed,encoder_background,policy_background,encoder_seed,policy_seed,encoder_env,policy_env,score,episode_length,algorithm,clustering_time
59,4,multicolor,multicolor,4,3,CarRacing-v2,CarRacing-v2,-93.629555,999.0,ppo,0
60,1,multicolor,multicolor,4,4,CarRacing-v2,CarRacing-v2,878.161255,999.0,ppo,0
61,2,multicolor,multicolor,4,4,CarRacing-v2,CarRacing-v2,878.161255,999.0,ppo,0
62,3,multicolor,multicolor,4,4,CarRacing-v2,CarRacing-v2,878.161255,999.0,ppo,0
63,4,multicolor,multicolor,4,4,CarRacing-v2,CarRacing-v2,878.161255,999.0,ppo,0


In [5]:
"""
file has the following columns:
track_seed,encoder_background,policy_background,score,max_score_reached,episode_length,algorithm
"""

'\nfile has the following columns:\ntrack_seed,encoder_background,policy_background,score,max_score_reached,episode_length,algorithm\n'

# CarRacing

In [6]:
# Read the CSV file into a DataFrame
env_id_encoder = "CarRacing-v2"
env_id = "CarRacing-v2-multicolor"
encoder_order = ['green', 'red', 'blue'] if env_id_encoder.startswith("CarRacing-v2") else ['plain', 'green', 'red']
if env_id.endswith("multicolor"):
    encoder_order = ['multicolor']
stitching_methods = ["absolute", "relative", "translate"]
stitching_method = stitching_methods[0]
anchoring_method = "fps"
anchors_alpha = 0.999

a_alpha = str(anchors_alpha)

model_activation = "relu"
model_algo = "ppo"


stitch_filename = f"experiments/stitching_tests/{env_id}/{env_info}/{stitching_method}/"
if stitching_method == "translate":
    stitch_filename += f"{anchoring_method}/"
if stitching_method == "relative":
    stitch_filename += f"a_{a_alpha}/"
stitch_filename += f"{env_id_encoder}_{model_activation}_stitching_results_{model_algo}.csv"

stitch_filename = "experiments/stitching_tests/CarRacing-v2-multicolor/rgb/absolute/green-bg_CarRacing-v2_relu_stitching_results_ppo.csv"

print(stitch_filename, "\n")

df = pd.read_csv(stitch_filename)

# Convert 'encoder_background' to categorical with specified order
df['encoder_background'] = pd.Categorical(df['encoder_background'], categories=encoder_order, ordered=True)

# Convert 'policy_background' to categorical with specified order
df['policy_background'] = pd.Categorical(df['policy_background'], categories=encoder_order, ordered=True)

# df.groupby(['encoder_background', 'policy_background']).agg({'max_score_reached': ['mean', 'std']})
# show another column "score" next to max_score columnm containing the mean and std of the score
# df.groupby(['encoder_background', 'policy_background']).agg({'max_score_reached': ['mean', 'std'], 'score': ['mean', 'std']})

df.groupby(['encoder_background', 'policy_background']).agg({'score': ['mean', 'std']})


experiments/stitching_tests/CarRacing-v2-multicolor/rgb/absolute/green-bg_CarRacing-v2_relu_stitching_results_ppo.csv 



/var/folders/qk/b7b3_k9j55jd9gm10z0chm3m0000gn/T/ipykernel_7897/676842021.py:41: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['encoder_background', 'policy_background']).agg({'score': ['mean', 'std']})


score            
                                            mean         std
encoder_background policy_background                        
multicolor         multicolor         159.702141  400.574593

In [7]:
# get encoder seed and policy seed that have top average score for encoder_background=green and policy_background=red
df[(df['encoder_background'] == 'green') & (df['policy_background'] == 'red')].sort_values(by='score', ascending=False).head(1)

,env_seed,encoder_background,policy_background,encoder_seed,policy_seed,encoder_env,policy_env,score,episode_length,algorithm,clustering_time


In [8]:
df[(df['encoder_background'] == 'green') & (df['policy_background'] == 'red')].sort_values(by='score', ascending=True).head(1)

,env_seed,encoder_background,policy_background,encoder_seed,policy_seed,encoder_env,policy_env,score,episode_length,algorithm,clustering_time


In [9]:
# print the total number of rows
print(f"Total number of rows: {df.shape[0]}")

Total number of rows: 64


In [10]:
# show the mean score of all the experiments where encoder_background == policy_background
# compute the mean score
print('Same encoder and policy background')
print(df[df['encoder_background'] == df['policy_background']]['score'].mean())
print(df[df['encoder_background'] == df['policy_background']]['score'].std())
print()
# show the mean score of all the experiments, excluding where encoder_background == policy_background
# compute the mean score
print('Different encoder and policy background')
print(df[df['encoder_background'] != df['policy_background']]['score'].mean())
print(df[df['encoder_background'] != df['policy_background']]['score'].std())

print('Cumulative scores')
print(df['score'].mean())
print(df['score'].std())

# get scores for when encoder_seed == policy_seed and encoder_background == policy_background
print('Same encoder and policy seed')
print(df[(df['encoder_seed'] == df['policy_seed']) & (df['encoder_background'] == df['policy_background'])]['score'].mean())
print(df[(df['encoder_seed'] == df['policy_seed']) & (df['encoder_background'] == df['policy_background'])]['score'].std())

# get all the rows where encoder_seed == policy_seed and encoder_background == policy_background
equal_seed = df[(df['encoder_seed'] == df['policy_seed']) & (df['encoder_background'] == df['policy_background'])]
# compute aggregate score over environment seeds, show mean and std, encoder_seed and policy_seed, encoder_background and policy_background only for values that are the not nan
equal_seed_scores = equal_seed.groupby(['encoder_seed', 'policy_seed', 'encoder_background', 'policy_background']).agg({'score': ['mean', 'std']}).dropna()
# rename mean score column to mean_score
equal_seed_scores.columns = ['mean_score', 'std_score']
# aggregate encoder_seed and policy_seed into one column called seed
equal_seed_scores['seed'] = equal_seed_scores.index.get_level_values('encoder_seed').astype(str)
# disaggregate encoder_seed and policy_seed, preserve encoder_background and policy_background
equal_seed_scores.index = equal_seed_scores.index.droplevel(['encoder_seed', 'policy_seed'])
# compute mean and std over seeds

equal_seed_scores

Same encoder and policy background
159.7021405994892
400.57459315099624

Different encoder and policy background
nan
nan
Cumulative scores
159.7021405994892
400.57459315099624
Same encoder and policy seed
845.2463531494141
24.709410023336787


/var/folders/qk/b7b3_k9j55jd9gm10z0chm3m0000gn/T/ipykernel_7897/2909045596.py:25: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  equal_seed_scores = equal_seed.groupby(['encoder_seed', 'policy_seed', 'encoder_background', 'policy_background']).agg({'score': ['mean', 'std']}).dropna()


mean_score  std_score seed
encoder_background policy_background                            
multicolor         multicolor         831.140076        0.0    1
                   multicolor         815.466553        0.0    2
                   multicolor         856.217529        0.0    3
                   multicolor         878.161255        0.0    4

When encoder seed and policy seed are different, or when the encoder and policy background are different

In [11]:
# get all the rows where encoder_seed != policy_seed or encoder_background != policy_background
equal_seed = df[(df['encoder_seed'] != df['policy_seed']) | (df['encoder_background'] != df['policy_background'])]
# compute aggregate score over environment seeds, show mean and std, encoder_seed and policy_seed, encoder_background and policy_background only for values that are the not nan
diff_scores = equal_seed.groupby(['encoder_seed', 'policy_seed', 'encoder_background', 'policy_background']).agg({'score': ['mean', 'std']}).dropna()
# rename mean score column to mean_score
diff_scores.columns = ['mean_score', 'std_score']
# aggregate encoder_seed and policy_seed into one column called seed
diff_scores['seed'] = diff_scores.index.get_level_values('encoder_seed').astype(str)
# disaggregate encoder_seed and policy_seed, preserve encoder_background and policy_background
diff_scores.index = diff_scores.index.droplevel(['encoder_seed', 'policy_seed'])
# compute mean and std over seeds

diff_scores.groupby(['encoder_background', 'policy_background']).agg({'mean_score': ['mean', 'std']})


/var/folders/qk/b7b3_k9j55jd9gm10z0chm3m0000gn/T/ipykernel_7897/2418010615.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  diff_scores = equal_seed.groupby(['encoder_seed', 'policy_seed', 'encoder_background', 'policy_background']).agg({'score': ['mean', 'std']}).dropna()
/var/folders/qk/b7b3_k9j55jd9gm10z0chm3m0000gn/T/ipykernel_7897/2418010615.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  diff_scores.groupby(['encoder_background', 'policy_background']).agg({'mean_score': ['mean', 'std']})


mean_score           
                                           mean        std
encoder_background policy_background                      
multicolor         multicolor        -68.812597  40.948082

Compute mean scores over different seeds for when encoder_background and policy_background are the same, and the seed too

In [12]:
equal_background_scores = equal_seed_scores[equal_seed_scores.index.get_level_values('encoder_background') == equal_seed_scores.index.get_level_values('policy_background')]
average_scores = equal_background_scores.groupby(['encoder_background', 'policy_background']).agg({'mean_score': 'mean', 'std_score': 'std'})
average_scores

/var/folders/qk/b7b3_k9j55jd9gm10z0chm3m0000gn/T/ipykernel_7897/2295913214.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  average_scores = equal_background_scores.groupby(['encoder_background', 'policy_background']).agg({'mean_score': 'mean', 'std_score': 'std'})


,,mean_score,std_score
encoder_background,policy_background,,
multicolor,multicolor,845.246353,0.0


In [13]:
median_scores = equal_background_scores.groupby(['encoder_background', 'policy_background']).agg({'mean_score': 'median', 'std_score': 'std'})
median_scores


/var/folders/qk/b7b3_k9j55jd9gm10z0chm3m0000gn/T/ipykernel_7897/790856185.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  median_scores = equal_background_scores.groupby(['encoder_background', 'policy_background']).agg({'mean_score': 'median', 'std_score': 'std'})


,,mean_score,std_score
encoder_background,policy_background,,
multicolor,multicolor,843.678802,0.0


Get maximum scores

In [14]:
# save idxmax to a list
idxmax_list = df[(df['encoder_seed'] == df['policy_seed']) & (df['encoder_background'] == df['policy_background'])].groupby(['encoder_background']).agg({'score': ['max', 'mean', 'std', 'idxmax']})['score']['idxmax'].tolist()
print(idxmax_list)
# select encoder_background, policy_background, seeds from the list
selection = df.iloc[idxmax_list][['encoder_background', 'policy_background', 'encoder_seed', 'policy_seed', 'score']]
selection

[60]


/var/folders/qk/b7b3_k9j55jd9gm10z0chm3m0000gn/T/ipykernel_7897/128057864.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  idxmax_list = df[(df['encoder_seed'] == df['policy_seed']) & (df['encoder_background'] == df['policy_background'])].groupby(['encoder_background']).agg({'score': ['max', 'mean', 'std', 'idxmax']})['score']['idxmax'].tolist()


,encoder_background,policy_background,encoder_seed,policy_seed,score
60,multicolor,multicolor,4,4,878.161255


In [15]:
stitching_method = stitching_methods[2]
anchoring_method = "fps"

model_activation = "relu"
model_algo = "ppo"

stitch_filename = f"experiments/stitching_tests/{env_id}/{env_info}/{stitching_method}/"
if stitching_method == "translate":
    stitch_filename += f"{anchoring_method}/"
stitch_filename += f"{env_id_encoder}_{model_activation}_stitching_results_{model_algo}.csv"

df_transl = pd.read_csv(stitch_filename)

FileNotFoundError: [Errno 2] No such file or directory: 'experiments/stitching_tests/CarRacing-v2-multicolor/rgb/translate/fps/CarRacing-v2_relu_stitching_results_ppo.csv'

In [ ]:
# select all the cells where encoder_seed == selection.encoder_seed and policy_seed == selection.policy_seed, for each row in selection
selection1 = df_transl[(df_transl['encoder_seed'] == selection.iloc[0]['encoder_seed']) & (df_transl['policy_seed'] == selection.iloc[0]['policy_seed'])]
selection2 = df_transl[(df_transl['encoder_seed'] == selection.iloc[1]['encoder_seed']) & (df_transl['policy_seed'] == selection.iloc[1]['policy_seed'])]
selection3 = df_transl[(df_transl['encoder_seed'] == selection.iloc[2]['encoder_seed']) & (df_transl['policy_seed'] == selection.iloc[2]['policy_seed'])]
selection4 = df_transl[(df_transl['encoder_seed'] == selection.iloc[3]['encoder_seed']) & (df_transl['policy_seed'] == selection.iloc[3]['policy_seed'])]


# CarRacing multicolor

In [ ]:
# Read the CSV file into a DataFrame
background = "multicolor"
env_id_encoder = "CarRacing-v2-multicolor"
env_id = "CarRacing-v2"
encoder_order = ['multicolor'] if env_id_encoder.startswith("CarRacing-v2") else ['plain', 'green', 'red']
controller_order = ['green', 'red', 'blue'] if env_id_encoder.startswith("CarRacing-v2") else ['plain', 'green', 'red']
stitching_methods = ["absolute", "relative", "translate"]
stitching_method = stitching_methods[0]
anchoring_method = "fps"
anchors_alpha = 0.999

a_alpha = str(anchors_alpha)

model_activation = "relu"
model_algo = "ppo"


stitch_filename = f"experiments/stitching_tests/{env_id}/{env_info}/{stitching_method}/"
if stitching_method == "translate":
    stitch_filename += f"{anchoring_method}/"
if stitching_method == "relative":
    stitch_filename += f"a_{a_alpha}/"
# stitch_filename += f"{background}-bg_{env_id_encoder}_{model_activation}_stitching_results_{model_algo}.csv"
stitch_filename += f"{env_id_encoder}_{model_activation}_stitching_results_{model_algo}.csv"

df = pd.read_csv(stitch_filename)

# Convert 'encoder_background' to categorical with specified order
df['encoder_background'] = pd.Categorical(df['encoder_background'], categories=encoder_order, ordered=True)

# Convert 'policy_background' to categorical with specified order
df['policy_background'] = pd.Categorical(df['policy_background'], categories=controller_order, ordered=True)

# df.groupby(['encoder_background', 'policy_background']).agg({'max_score_reached': ['mean', 'std']})
# show another column "score" next to max_score columnm containing the mean and std of the score
# df.groupby(['encoder_background', 'policy_background']).agg({'max_score_reached': ['mean', 'std'], 'score': ['mean', 'std']})

df.groupby(['encoder_background', 'policy_background']).agg({'score': ['mean', 'std']})


In [ ]:
# print results when encoder_seed and policy_seed are the same
print('Same encoder and policy seed')
print(df[(df['encoder_seed'] == df['policy_seed'])]['score'].mean())
print(df[(df['encoder_seed'] == df['policy_seed'])]['score'].std())

# print results when encoder_seed and policy_seed are different
print('Different encoder and policy seed')
print(df[(df['encoder_seed'] != df['policy_seed'])]['score'].mean())
print(df[(df['encoder_seed'] != df['policy_seed'])]['score'].std())

# print cumulative results
print('Cumulative scores')
print(df['score'].mean())
print(df['score'].std())

# ATARI

In [ ]:
# Read the CSV file into a DataFrame
env_id = "BoxingNoFrameskip-v4"
env_id_encoder = "BoxingNoFrameskip-v4"
encoder_order = ['plain', 'green', 'red']
stitching_methods = ["absolute", "relative", "translate"]
stitching_method = stitching_methods[2]
anchoring_method = "fps"

anchors_alpha = 0.999

a_alpha = str(anchors_alpha)

model_activation = "relu"
model_algo = "ppo"

stitch_filename = f"experiments/stitching_tests/{env_id}/{env_info}/{stitching_method}/"
if stitching_method == "translate":
    stitch_filename += f"{anchoring_method}/"
if stitching_method == "relative":
    stitch_filename += f"a_{a_alpha}/"
stitch_filename += f"{env_id_encoder}_{model_activation}_stitching_results_{model_algo}.csv"

df = pd.read_csv(stitch_filename)

# Convert 'encoder_background' to categorical with specified order
df['encoder_background'] = pd.Categorical(df['encoder_background'], categories=encoder_order, ordered=True)

# Convert 'policy_background' to categorical with specified order
df['policy_background'] = pd.Categorical(df['policy_background'], categories=encoder_order, ordered=True)

# df.groupby(['encoder_background', 'policy_background']).agg({'max_score_reached': ['mean', 'std']})
# show another column "score" next to max_score columnm containing the mean and std of the score
# df.groupby(['encoder_background', 'policy_background']).agg({'max_score_reached': ['mean', 'std'], 'score': ['mean', 'std']})

df.groupby(['encoder_background', 'policy_background']).agg({'score': ['mean', 'std']})


In [ ]:
# print the dataframe where encoder_background == policy_background and encoder_background == 'plain'. Show the score, encoder_seed and policy_seed
df[(df['policy_background'] == 'plain') & (df['encoder_background'] == 'plain')][['score', 'encoder_background', 'policy_background', 'encoder_seed', 'policy_seed']]

In [ ]:
# print the total number of rows
print(f"Total number of rows: {df.shape[0]}")

# get scores for when encoder_seed == policy_seed and encoder_background == policy_background
print('Same encoder and policy seed')
print(df[(df['encoder_seed'] == df['policy_seed']) & (df['encoder_background'] == df['policy_background'])]['score'].mean())
print(df[(df['encoder_seed'] == df['policy_seed']) & (df['encoder_background'] == df['policy_background'])]['score'].std())

# show the mean score of all the experiments where encoder_background == policy_background
# compute the mean score
print('Same encoder and policy background')
print(df[df['encoder_background'] == df['policy_background']]['score'].mean())
print(df[df['encoder_background'] == df['policy_background']]['score'].std())
print()
# show the mean score of all the experiments, excluding where encoder_background == policy_background
# compute the mean score
print('Different encoder and policy background')
print(df[df['encoder_background'] != df['policy_background']]['score'].mean())
print(df[df['encoder_background'] != df['policy_background']]['score'].std())

print('Cumulative scores')
print(df['score'].mean())
print(df['score'].std())

# get scores for when encoder_seed == policy_seed and encoder_background == policy_background
print('Same encoder and policy seed')
print(df[(df['encoder_seed'] == df['policy_seed']) & (df['encoder_background'] == df['policy_background'])]['score'].mean())
print(df[(df['encoder_seed'] == df['policy_seed']) & (df['encoder_background'] == df['policy_background'])]['score'].std())

# get all the rows where encoder_seed == policy_seed and encoder_background == policy_background
equal_seed = df[(df['encoder_seed'] == df['policy_seed']) & (df['encoder_background'] == df['policy_background'])]
# compute mean score over colors, show mean and std, encoder_seed and policy_seed, encoder_background and policy_background only for values that are the not nan
equal_seed_scores = equal_seed.groupby(['encoder_background', 'policy_background']).agg({'score': ['mean', 'std']}).dropna()
equal_seed_scores

When encoder seed and policy seed are different, or when the encoder and policy background are different (in which case we do not care whether the seed is the same or not)

In [ ]:
# get all the rows where encoder_seed != policy_seed or encoder_background != policy_background
diff = df[(df['encoder_seed'] != df['policy_seed']) | (df['encoder_background'] != df['policy_background'])]
# compute aggregate score over environment seeds, show mean and std, encoder_seed and policy_seed, encoder_background and policy_background only for values that are the not nan
diff_scores = diff.groupby(['encoder_background', 'policy_background']).agg({'score': ['mean', 'std']}).dropna()
diff_scores


# ANALYSIS

In [ ]:
env_id = "CarRacing-v2"
env_info = "rgb"
anchoring_method = "fps"

""" SHOW LOGGED DATA """

# read stitching_results_relative_ppo with pandas
df1 = pd.read_csv(f'{test_path}/{env_id}/{env_info}/absolute/relu_stitching_results_ppo.csv', sep=',')
df1.head()
stitch_filename_2 = f"experiments/stitching_tests/{env_id}/{env_info}/{stitching_method}/"
if stitching_method == "translate":
    stitch_filename_2 += f"{anchoring_method}/"
stitch_filename_2 += f"{model_activation}_stitching_results_{model_algo}.csv"
# df2 = pd.read_csv(f'{test_path}/{env_id}/{env_info}/translate/relu_stitching_results_ppo.csv', sep=',')
df2 = pd.read_csv(stitch_filename_2)



# join the two dataframes adding a new column called "method". Use "absolute" under method for df1 and "translate" for df2
df1['method'] = 'absolute'
# sum 40 to df1 env_seed column
df1['env_seed'] = df1['env_seed'] + 39
df2['method'] = 'translate'

# create df3 containing data where encoder_background and policy_background are the same, and remove it from df1 and df2
df3 = df1[df1['encoder_background'] == df1['policy_background']]
df3['method'] = 'absolute_original'
df1 = df1[df1['encoder_background'] != df1['policy_background']]
df4 = df2[df2['encoder_background'] == df2['policy_background']]
df4['method'] = 'translate_original'
df2 = df2[df2['encoder_background'] != df2['policy_background']]

# concatenate the four dataframes
df = pd.concat([df1, df2, df3, df4])
df.head()


In [ ]:
import plotly.express as px
import plotly.graph_objects as go

""" df contains the following columns: env_seed	encoder_background	policy_background	score	max_score_reached	episode_length	algorithm
Create a histogram with x axis being the env_seed and y axis being the score. Use "absolute" and "translation" as colors.
Each score must be a bar.
"""
fig = px.box(df, x="env_seed", y="score", color="method", facet_row="encoder_background", height=800)
#fig = px.histogram(df, x="env_seed", y="score", color="method", barmode="group", histfunc="avg")
fig.show()


In [ ]:
Allora puoi fare un istrogramma con plotly express. Prendi in input il df e metti come x il track_seed,
come y il valore di score raggiunto o quello che è, come color il metodo usato (che sarà o absolute o translation)